<a href="https://colab.research.google.com/github/mokshagna28/mokshagna/blob/main/MeetMate_AI_Agent_for_Smarter_Online_Meetings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai-whisper gradio transformers torch
!sudo apt update && sudo apt install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (238 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as reposit

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload logo.png, transcript.png, summary.png, actions.png


Saving 1747720284147890326796l836-voicemaker.in-speech.mp3 to 1747720284147890326796l836-voicemaker.in-speech (1).mp3


In [ ]:
simport whisper
from transformers import pipeline
import re
import gradio as gr
from PIL import Image
import os

whisper_model = whisper.load_model("base")


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


def transcribe_audio(audio_path):
    try:
        result = whisper_model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        return f"[Error in transcription]: {str(e)}"

def summarize_text(text):
    try:
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        summaries = [summarizer(chunk)[0]['summary_text'] for chunk in chunks]
        return " ".join(summaries)
    except Exception as e:
        return f"[Error in summarization]: {str(e)}"

def extract_action_items(text):
    try:
        actions = re.findall(r"(?:[A-Z][^.]*\bwill\b[^.]*\.)", text)
        return "\n".join(actions) if actions else "No clear action items detected."
    except Exception as e:
        return f"[Error extracting action items]: {str(e)}"

def process_meeting_audio(audio_path):
    transcript = transcribe_audio(audio_path)
    summary = summarize_text(transcript)
    actions = extract_action_items(transcript)
    return transcript, summary, actions


def safe_load_image(path, fallback_text):
    try:
        return Image.open(path)
    except:
        print(f"⚠️ Couldn't load image: {path}")
        return None

logo = safe_load_image("/content/logo.png", "MeetMate")
img_transcript = safe_load_image("/content/transcription.webp", "Transcript")
img_summary = safe_load_image("/content/summary.png", "Summary")
img_actions = safe_load_image("/content/action.jpg", "Action Items")


def handle_audio(audio_path):
    transcript, summary, actions = process_meeting_audio(audio_path)
    return img_transcript, transcript, img_summary, summary, img_actions, actions

with gr.Blocks() as demo:
    if logo:
        gr.Image(value=logo, show_label=False)
    gr.Markdown("## 🎤 Upload Your Meeting Audio (.mp3/.wav)")

    # Removed the 'source' argument as it's not accepted by the constructor
    audio_input = gr.Audio(type="filepath", label="Meeting Audio")
    process_button = gr.Button("🚀 Process Meeting")

    with gr.Row():
        out_img_1 = gr.Image()
        out_transcript = gr.Textbox(label="📝 Transcript", lines=6)

    with gr.Row():
        out_img_2 = gr.Image()
        out_summary = gr.Textbox(label="📄 Summary", lines=4)

    with gr.Row():
        out_img_3 = gr.Image()
        out_actions = gr.Textbox(label="✅ Action Items", lines=4)

    process_button.click(fn=handle_audio, inputs=audio_input,
                         outputs=[
                             out_img_1, out_transcript,
                             out_img_2, out_summary,
                             out_img_3, out_actions
                         ])

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1872dc79ca11f2e7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
image_input = gr.Image(label="Optional Image to Include in Email (e.g., Team Photo)")


In [ ]:
image = None # Placeholder: Set this to None or a sample PIL Image object for testing


if image is not None:
    import base64
    import io
    buffered = io.BytesIO()
    # Ensure the image object is a PIL Image instance before saving
    if isinstance(image, Image.Image):
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
        image_html = f'<img src="data:image/png;base64,{img_str}" width="400"/>'
    else:
        # Handle cases where 'image' might be a path or other type if not loaded correctly
        print("Warning: 'image' variable is not a PIL Image object. Skipping image embedding.")
        image_html = ""
else:
    image_html = ""

# You can now print image_html to see the result (either the image tag or empty string)
print(image_html)


In [ ]:
body = f"""
<h2>📋 MeetMate – Smart Meeting Report</h2>
{image_html}
...
"""


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install -q openai-whisper gradio transformers torch
!sudo apt update && sudo apt install -y ffmpeg

# %%
from google.colab import files
uploaded = files.upload()  # Upload logo.png, transcript.png, summary.png, actions.png

# %%
import whisper
from transformers import pipeline
import re
import gradio as gr
from PIL import Image
import os

whisper_model = whisper.load_model("base")


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


def transcribe_audio(audio_path):
    try:
        result = whisper_model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        return f"[Error in transcription]: {str(e)}"

def summarize_text(text):
    try:
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        summaries = [summarizer(chunk)[0]['summary_text'] for chunk in chunks]
        return " ".join(summaries)
    except Exception as e:
        return f"[Error in summarization]: {str(e)}"

def extract_action_items(text):
    try:
        actions = re.findall(r"(?:[A-Z][^.]*\bwill\b[^.]*\.)", text)
        return "\n".join(actions) if actions else "No clear action items detected."
    except Exception as e:
        return f"[Error extracting action items]: {str(e)}"

def process_meeting_audio(audio_path):
    transcript = transcribe_audio(audio_path)
    summary = summarize_text(transcript)
    actions = extract_action_items(transcript)
    return transcript, summary, actions


def safe_load_image(path, fallback_text):
    try:
        return Image.open(path)
    except:
        print(f"⚠️ Couldn't load image: {path}")
        return None

logo = safe_load_image("/content/logo.png", "MeetMate")
img_transcript = safe_load_image("/content/transcription.webp", "Transcript")
img_summary = safe_load_image("/content/summary.png", "Summary")
img_actions = safe_load_image("/content/action.jpg", "Action Items")


# Modified handle_audio to also return the audio_path
def handle_audio(audio_path):
    if audio_path is None:
        return None, "", None, "", None, "", None
    transcript, summary, actions = process_meeting_audio(audio_path)
    return img_transcript, transcript, img_summary, summary, img_actions, actions, audio_path

# Added a new function to handle the email creation
def create_email_attachment(audio_path):
    if audio_path is None or not os.path.exists(audio_path):
        print("No valid audio file path available for attachment.")
        return None

    from email.mime.base import MIMEBase
    from email import encoders
    import os

    filename = os.path.basename(audio_path)
    try:
        with open(audio_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f"attachment; filename= {filename}")
            # In a real email scenario, you would attach 'part' to a 'message' object here
            print(f"Successfully created attachment part for {filename}")
            # Return the attachment part (or handle attaching to a message object)
            return part
    except Exception as e:
        print(f"[Error creating attachment]: {str(e)}")
        return None


with gr.Blocks() as demo:
    # Use gr.State to store the audio_path
    audio_file_path_state = gr.State(None)

    if logo:
        gr.Image(value=logo, show_label=False)
    gr.Markdown("## 🎤 Upload Your Meeting Audio (.mp3/.wav)")

    audio_input = gr.Audio(type="filepath", label="Meeting Audio")
    process_button = gr.Button("🚀 Process Meeting")

    with gr.Row():
        out_img_1 = gr.Image()
        out_transcript = gr.Textbox(label="📝 Transcript", lines=6)

    with gr.Row():
        out_img_2 = gr.Image()
        out_summary = gr.Textbox(label="📄 Summary", lines=4)

    with gr.Row():
        out_img_3 = gr.Image()
        out_actions = gr.Textbox(label="✅ Action Items", lines=4)

    process_button.click(fn=handle_audio, inputs=audio_input,
                         outputs=[
                             out_img_1, out_transcript,
                             out_img_2, out_summary,
                             out_img_3, out_actions,
                             audio_file_path_state
                         ])


demo.launch()

image_input = gr.Image(label="Optional Image to Include in Email (e.g., Team Photo)")

image = None


if image is not None:
    import base64
    import io
    buffered = io.BytesIO()

    if isinstance(image, Image.Image):
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
        image_html = f'<img src="data:image/png;base64,{img_str}" width="400"/>'
    else:

        print("Warning: 'image' variable is not a PIL Image object. Skipping image embedding.")
        image_html = ""
else:
    image_html = ""


print(image_html)


body = f"""
<h2>📋 MeetMate – Smart Meeting Report</h2>
{image_html}
...
"""


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 261 kB in 1s (234 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' 

Saving M_0874_15y5m_1.wav to M_0874_15y5m_1.wav


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://53c272536fa079f868.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import whisper
from transformers import pipeline
import re
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

# Load models
whisper_model = whisper.load_model("base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def transcribe_and_summarize(audio, sender_email, app_password, recipients):
    # Transcription
    # When type="filepath", the 'audio' object provided by Gradio has a 'name' attribute which is the path to the temporary file.
    audio_path = audio.name
    result = whisper_model.transcribe(audio_path)
    transcript = result["text"]

    # Summarization (chunking if needed)
    max_chunk = 1000
    transcript_chunks = [transcript[i:i+max_chunk] for i in range(0, len(transcript), max_chunk)]
    summary = ""
    for chunk in transcript_chunks:
        summary_text = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        summary += summary_text + "\n"

    # Action Item Extraction
    action_items = re.findall(r'\b[A-Z][a-z]+\b.*?\b(will|needs to|should)\b.*?\.', transcript)
    action_list = [item.strip() for item in action_items]

    # Send email
    send_meeting_email(sender_email, app_password, recipients, "MeetMate Report", transcript, summary, action_list, audio_path)

    return transcript, summary, action_list

def send_meeting_email(sender_email, app_password, recipient_list, subject, transcript, summary, action_items, audio_path):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ", ".join(recipient_list)
    message['Subject'] = subject

    body = f"""
    <h2>📋 MeetMate – Smart Meeting Report</h2>
    <h3>🗒️ Summary:</h3>
    <p>{summary}</p>
    <h3>📝 Transcript:</h3>
    <p>{transcript}</p>
    <h3>✅ Action Items:</h3>
    <ul>
    {''.join(f"<li>{item}</li>" for item in action_items)}
    </ul>
    <p>Thanks,<br>MeetMate AI Assistant</p>
    """

    message.attach(MIMEText(body, 'html'))

    # Attach audio file
    try:
        filename = os.path.basename(audio_path)
        with open(audio_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f"attachment; filename= {filename}")
        message.attach(part)
    except Exception as e:
        print(f"❌ Error attaching file: {e}")

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, recipient_list, message.as_string())
        server.quit()
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

with gr.Blocks() as demo:
    gr.Markdown("""# 🤖 MeetMate – AI Agent for Smarter Online Meetings
Upload a meeting recording and get the Transcript, Summary, Action Items, and Email Report!""")

    with gr.Row():
        # Changed type="file" to type="filepath"
        audio_input = gr.Audio(label="/content/M_0874_15y5m_1.mp4", type="filepath")

    with gr.Row():
        email_input = gr.Text(label="Sender Gmail")
        password_input = gr.Text(label="App Password", type="password")
        recipient_input = gr.Text(label="Recipient Emails (comma-separated)")

    with gr.Row():
        btn = gr.Button("Process and Email Report")

    with gr.Row():
        transcript_output = gr.Textbox(label="📄 Transcript")
        summary_output = gr.Textbox(label="📝 Summary")
        action_output = gr.Textbox(label="✅ Action Items")

    btn.click(fn=lambda audio, sender, pwd, recs: transcribe_and_summarize(
                audio, sender, pwd, [email.strip() for email in recs.split(',')]),
              inputs=[audio_input, email_input, password_input, recipient_input],
              outputs=[transcript_output, summary_output, action_output])

# Launch the app
demo.launch()

Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57c8f3ee4b4b072ae0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import whisper
from transformers import pipeline
import re
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import openai

# Load models
whisper_model = whisper.load_model("base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
openai.api_key = "your-openai-api-key"  # Replace with your actual OpenAI API key

# Store transcript globally
stored_transcript = ""

def transcribe_and_summarize(audio, sender_email, app_password, recipients):
    global stored_transcript
    # Transcription
    audio_path = audio.name
    result = whisper_model.transcribe(audio_path)
    transcript = result["text"]
    stored_transcript = transcript  # Store for Q&A

    # Summarization (chunking if needed)
    max_chunk = 1000
    transcript_chunks = [transcript[i:i+max_chunk] for i in range(0, len(transcript), max_chunk)]
    summary = ""
    for chunk in transcript_chunks:
        summary_text = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        summary += summary_text + "\n"

    # Action Item Extraction
    action_items = re.findall(r'\b[A-Z][a-z]+\b.*?\b(will|needs to|should)\b.*?\.', transcript)
    action_list = [item.strip() for item in action_items]

    # Send email
    send_meeting_email(sender_email, app_password, recipients, "MeetMate Report", transcript, summary, action_list, audio_path)

    return transcript, summary, action_list

def send_meeting_email(sender_email, app_password, recipient_list, subject, transcript, summary, action_items, audio_path):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ", ".join(recipient_list)
    message['Subject'] = subject

    body = f"""
    <h2>📋 MeetMate – Smart Meeting Report</h2>
    <h3>🗒️ Summary:</h3>
    <p>{summary}</p>
    <h3>📝 Transcript:</h3>
    <p>{transcript}</p>
    <h3>✅ Action Items:</h3>
    <ul>
    {''.join(f"<li>{item}</li>" for item in action_items)}
    </ul>
    <p>Thanks,<br>MeetMate AI Assistant</p>
    """

    message.attach(MIMEText(body, 'html'))

    try:
        filename = os.path.basename(audio_path)
        with open(audio_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f"attachment; filename= {filename}")
        message.attach(part)
    except Exception as e:
        print(f"❌ Error attaching file: {e}")

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, recipient_list, message.as_string())
        server.quit()
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

def answer_question(question):
    global stored_transcript
    if not stored_transcript:
        return "Please upload and process a meeting audio first."

    prompt = f"Meeting transcript:\n{stored_transcript}\n\nQ: {question}\nA:"
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response["choices"][0]["message"]["content"]
        return answer.strip()
    except Exception as e:
        return f"❌ Error answering question: {e}"

with gr.Blocks() as demo:
    gr.Markdown("""# 🤖 MeetMate – AI Agent for Smarter Online Meetings
Upload a meeting recording and get the Transcript, Summary, Action Items, and Email Report!""")

    with gr.Row():
        # Changed type="file" to type="filepath"
        audio_input = gr.Audio(label="/content/M_0874_15y5m_1.mp4", type="filepath")

    with gr.Row():
        email_input = gr.Text(label="Sender Gmail")
        password_input = gr.Text(label="App Password", type="password")
        recipient_input = gr.Text(label="Recipient Emails (comma-separated)")

    with gr.Row():
        btn = gr.Button("Process and Email Report")

    with gr.Row():
        transcript_output = gr.Textbox(label="📄 Transcript")
        summary_output = gr.Textbox(label="📝 Summary")
        action_output = gr.Textbox(label="✅ Action Items")

    with gr.Row():
        question_input = gr.Textbox(label="Ask a Question about the Meeting")
        answer_output = gr.Textbox(label="🤖 Answer")
        ask_btn = gr.Button("Get Answer")

    btn.click(fn=lambda audio, sender, pwd, recs: transcribe_and_summarize(
                audio, sender, pwd, [email.strip() for email in recs.split(',')]),
              inputs=[audio_input, email_input, password_input, recipient_input],
              outputs=[transcript_output, summary_output, action_output])

    ask_btn.click(fn=answer_question, inputs=question_input, outputs=answer_output)

# Launch the app
demo.launch()

Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://480d93b36abf2b2e36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
